In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#!ls "/content/gdrive/MyDrive/"

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.utils.data as Data
from torchvision import datasets, transforms
import numpy as np
import os
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np

class Model_Drop(nn.Module):
    def __init__(self):

        super(Model_Drop, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3,padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3,padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3,padding=1)
        self.fc1 = nn.Linear(7*7*64, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 10)
        self.drop_layer = nn.Dropout(p=0.50)

    def last_hidden_layer_output(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(F.relu(self.conv4(x)), 2)
        x = x.view(-1, 7*7*64)
        x = self.drop_layer((F.relu(self.fc1(x))))
        x = self.drop_layer((F.relu(self.fc2(x))))
        return x

    def forward(self, x):
        x = self.last_hidden_layer_output(x)
        x = self.fc3(x)
        return x


In [ ]:
!git clone https://github.com/knamdar/data.git

fatal: destination path 'data' already exists and is not an empty directory.


In [ ]:
mnist_train = datasets.MNIST("data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("data", train=False, download=True, transform=transforms.ToTensor())

trainload = DataLoader(mnist_train, batch_size=100, shuffle=True)
testload = DataLoader(mnist_test, batch_size=100, shuffle=False)


In [ ]:
teach_model = Model_Drop()
teach_model.to(device)
teach_model.load_state_dict(torch.load("/content/gdrive/MyDrive/MNIST_DIGIT_model_cnn_teacher_T_20.pt"))
teach_model.eval()

Model_Drop(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
  (drop_layer): Dropout(p=0.5, inplace=False)
)

In [ ]:
model = Model_Drop()
model.to(device)

Model_Drop(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=3136, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
  (drop_layer): Dropout(p=0.5, inplace=False)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
#criterion2 = nn.KLDivLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.5)

torch.manual_seed(2)
torch.cuda.manual_seed(2)

In [ ]:
model.train()
T = 20

def dist_loss(t, s, temp):
    prob_t = F.softmax(t/temp, dim=1)
    log_prob_s = F.log_softmax(s/temp, dim=1)
    dist_loss = -(prob_t*log_prob_s).sum(dim=1).mean()
    return dist_loss

for epoch in range(20):

    model.train()
    loss_sigma = 0.0
    correct = 0.0
    total = 0.0
    for i, data in enumerate(trainload):

        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs.float())

        teacher_outputs = teach_model(inputs.float())

        loss = dist_loss(teacher_outputs, outputs, T)

        loss.backward()

        optimizer.step()


Current epoch is :  0
Current epoch is :  1
Current epoch is :  2
Current epoch is :  3
Current epoch is :  4
Current epoch is :  5
Current epoch is :  6
Current epoch is :  7
Current epoch is :  8
Current epoch is :  9
Current epoch is :  10
Current epoch is :  11
Current epoch is :  12
Current epoch is :  13
Current epoch is :  14
Current epoch is :  15
Current epoch is :  16
Current epoch is :  17
Current epoch is :  18
Current epoch is :  19


In [ ]:
total_loss, total_err = 0., 0.

model.eval()

for X, y in testload:
    X, y = X.to(device), y.to(device)

    outputs = model(X.float())

    total_err += (outputs.max(dim=1)[1] != y).sum().item()

print("total test error is ", total_err / len(testload.dataset))

total test error is  0.0059


In [ ]:
model.eval()
torch.save(model.state_dict(), "MNIST_DIGIT_model_cnn_student_T_20.pt")


In [ ]:
torch.save(model.state_dict(), "/content/gdrive/MyDrive/MNIST_DIGIT_model_cnn_student_T_20.pt")